# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"> **Hopsworks Feature Store** </span>
<span style="font-width:bold; font-size: 2rem; color:#333;">Part 03 - Batch Inference</span>

## <span style="color:#ff5f27;">📝 Imports</span>

In [3]:
import ast
import numpy as np
import pandas as pd
import hopsworks
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from ast import literal_eval
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import hsfs
import os
import joblib

## <span style="color:#ff5f27;">📡 Connecting to Hopsworks Feature Store</span>

In [4]:
# Login to Hopsworks
'''
project = hopsworks.login()
'''

project = hsfs.connection(
    host='staging.cloud.hopsworks.ai',                 # DNS of your Feature Store instance
    port=443,                           # Port to reach your Hopsworks instance, defaults to 443
    project='neo4j_tutorial',               # Name of your Hopsworks Feature Store project
    api_key_value='OzvDK7qnG4VlhPxX.3wzZfJWWFmuYegomtWDRXAAvvGhlJEPkg4bn9kSbQaWPDgC3akbtERQOojLtEl5r',            # The API key to authenticate with the feature store
    hostname_verification=True          # Disable for self-signed certificates
)

# Retrieve Feature Store
fs = project.get_feature_store()           # Get the project's default feature store

Connected. Call `.close()` to terminate connection gracefully.


## <span style="color:#ff5f27;">⚙️ Feature View Retrieval</span>

In [5]:
# Retrieve the churn_feature_view
feature_view = fs.get_feature_view(
        name = 'port_fv',
        version = 1,
)

## <span style="color:#ff5f27;">🗄 Model Registry</span>

In [6]:
# Get the model registry
mr = project.get_model_registry()

AttributeError: 'Connection' object has no attribute 'get_model_registry'

## <span style="color:#ff5f27;">🚀 Fetch the model</span>

In [20]:
# Retrieve the model from the model registry
retrieved_model = mr.get_model(
    name="neo4j_neural_network",
    version=1,
)

# Download the saved model files to a local directory
saved_model_dir = retrieved_model.download()

Finished: Reading data from Hopsworks, using Hive (22.24s) 


In [ ]:
# Load the saved model using joblib from the downloaded model directory
model = joblib.load(saved_model_dir + "/neural_network_model.pkl")

# Display the retrieved model
model

## <span style="color:#ff5f27;">🔮 Use trained model for prediction</span>

In [ ]:
# Combine embeddings and explode features
def combine_explode_embeddings(data):
    # Convert embeddings from String to List
    if type(data['source_node_embedding'].iloc[0]) == str:
        data['source_node_embedding'] = data['source_node_embedding'].apply(literal_eval)
    if type(data['target_node_embedding'].iloc[0]) == str:
        data['target_node_embedding'] = data['target_node_embedding'].apply(literal_eval)

    # Concatenate embeddings
    data['concatenated_embedding'] = data.source_node_embedding + data.target_node_embedding

    # Remove original embedding features
    data = data.drop(['source_node_embedding', 'target_node_embedding'], axis=1)

    # Convert to Numpy array
    data['concatenated_embedding'] = data['concatenated_embedding'].apply(np.array)

    # Explode Embedding features
    data_exploded = pd.DataFrame(
        data['concatenated_embedding'].to_list(),
        columns=[f'feature_{i}' for i in range(len(data.concatenated_embedding.iloc[0]))]
    )
    return data_exploded

In [ ]:
data = # load some nodes

data_exploded = combine_explode_embeddings(data)

model.predict(data_exploded)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the churn risk prediction tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai